In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [ ]:
pip install keras --user

Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'C:\python\Python37\python.exe -m pip install --upgrade pip' command.


In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model ,load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import tensorflow as tf

In [ ]:
from keras import models
from keras import layers
from keras.applications import VGG16
from keras import optimizers
from keras.layers.core import Flatten, Dense, Dropout, Lambda

In [ ]:

from __future__ import print_function, division
from builtins import range, input

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import InceptionV3,ResNet50,Xception
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cv2
from glob import glob
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
from numpy.random import seed
seed(8) #1
#from tensorflow import  set_random_seed
 #2

tf.random.set_seed(7)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os


In [ ]:
from os import listdir
data_list = listdir('D:/Minor/datanew')

#Delete some classes that may interfere


data_list

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:/datanew'

In [ ]:
DATASET_PATH  = 'E:/datanew/train'
test_dir =  'E:/datanew/test'
IMAGE_SIZE    = (150, 150)
NUM_CLASSES   = len(data_list)
BATCH_SIZE    = 10  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS    = 100
LEARNING_RATE = 0.001 #start off with high rate first 0.001 #5e-4

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=50,
                                   featurewise_center = True,
                                   featurewise_std_normalization = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.25,
                                   zoom_range=0.1,
                                   zca_whitening = True,
                                   channel_shift_range = 20,
                                   horizontal_flip = True ,
                                   vertical_flip = True ,
                                   validation_split = 0.2,
                                   fill_mode='constant')

In [ ]:
train_batches = train_datagen.flow_from_directory(DATASET_PATH,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "training",
                                                  seed=42,
                                                  class_mode="categorical",

                                                  )

In [ ]:

valid_batches = train_datagen.flow_from_directory(DATASET_PATH,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "validation",
                                                  seed=42,
                                                  class_mode="categorical",


                                                  )

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import initializers

In [ ]:
import keras.backend as k
k.clear_session() #Clear keras backend
try:
  os.mkdir('models')
except:
  pass
full_name='concatenate'
classes_number=3 #Number of classes
input_tensor=Input(shape=(150,150,3))
######################################################################################################
base_model1 = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)
features1 = base_model1.output
######################################################################################################
base_model2 = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)
features2 = base_model2.output
concatenated=layers.concatenate([features1,features2]) #Concatenate the extracted features
####################################################################################################
conv=layers.Conv2D(1024, (1, 1),padding='same')(concatenated) #add the concatenated features to a convolutional layer
feature = Flatten(name='flatten')(conv)
dp = Dropout(0.5)(feature) #add dropout
preds = Dense(classes_number, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.001))(dp)
Concatenated_model = Model(inputs=input_tensor, outputs=preds)
#######################################################
for layer in Concatenated_model.layers:
  layer.trainable = True
Concatenated_model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%full_name
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max') #creating checkpoint to save the best validation accuracy
callbacks_list = [checkpoint]

Concatenated_model.summary()
#Concatenated_model.fit_generator(train_batches, epochs=100,validation_data=valid_batches,shuffle=True,callbacks=callbacks_list) #start training

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

eval_generator = test_datagen.flow_from_directory(
        test_dir,target_size=IMAGE_SIZE,
        batch_size=1,
        shuffle=False,
        seed=42,


        class_mode="categorical")
eval_generator.reset()

Found 1419 images belonging to 3 classes.


In [ ]:
eval_generator.reset()
x = Concatenated_model.evaluate_generator(eval_generator,
                           steps = np.ceil(len(eval_generator) / BATCH_SIZE),
                           use_multiprocessing = False,
                           verbose = 1,
                           workers=1
                           )


print('Test loss:' , x[0])
print('Test accuracy:',x[1])

C:\Users\Prasoon Smirti\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '
142/142 [==============================] - 26s 106ms/step - loss: 0.2699 - accuracy: 0.9718
Test loss: 0.2698708474636078
Test accuracy: 0.9718309640884399


In [ ]:
preds = Concatenated_model.evaluate(eval_generator)
print ("Validation Loss = " + str(preds[0]))
print ("Validation Accuracy = " + str(preds[1]))

1419/1419 [==============================] - 159s 112ms/step - loss: 0.2602 - accuracy: 0.9457
Validation Loss = 0.26024168729782104
Validation Accuracy = 0.9457364082336426


In [ ]:
eval_generator.reset()
pred = Concatenated_model.predict(eval_generator,verbose=1)
print("Predictions finished")

1419/1419 [==============================] - 159s 109ms/step
Predictions finished


In [ ]:
pred_labels = np.argmax(pred, axis = 1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(classification_report(eval_generator.classes, pred_labels))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       267
           1       0.87      0.89      0.88       313
           2       0.96      0.96      0.96       839

    accuracy                           0.95      1419
   macro avg       0.94      0.94      0.94      1419
weighted avg       0.95      0.95      0.95      1419



In [ ]:
import seaborn as sns

In [ ]:
#con_mat = confusion_matrix(eval_generator.classes,pred_labels)
#plt.figure(figsize = (5,5))
#plt.title('CONFUSION MATRIX')
#sns.heatmap(con_mat, cmap='coolwarm',
#            yticklabels=['Negative', 'Positive'],
#            xticklabels=['Negative', 'Positive'],
#            annot=None)